In [1]:
!pip install trl
!pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
import torch
import transformers
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig,AutoModelForSeq2SeqLM, AutoModelForSequenceClassification
from trl import RewardTrainer, SFTTrainer , AutoModelForSeq2SeqLMWithValueHead,PPOTrainer, PPOConfig
from trl.core import LengthSampler
from trl import create_reference_model
from datasets import Dataset, load_dataset
import numpy as np
import json
import random
import pandas as pd
from transformers import Trainer, TrainingArguments
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead, create_reference_model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [4]:
dataset = load_dataset('CarperAI/openai_summarize_tldr')

In [5]:
dataset = dataset['train'].select(range(1000))

In [6]:
dataset

Dataset({
    features: ['prompt', 'label'],
    num_rows: 1000
})

In [7]:
dataset = dataset.rename_column("label", "response")

In [10]:
dataset

Dataset({
    features: ['prompt', 'response'],
    num_rows: 1000
})

In [11]:
### If we would have trained the 1st step(SFT), then that model will come here.

policy_model_id = "pszemraj/led-base-book-summary"
policy_model = AutoModelForSeq2SeqLM.from_pretrained(policy_model_id)
policy_model.to(device)
policy_tokenizer = AutoTokenizer.from_pretrained(policy_model_id)

In [12]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_model_id) 
ppo_model.to(device)

AutoModelForSeq2SeqLMWithValueHead(
  (pretrained_model): LEDForConditionalGeneration(
    (led): LEDModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): LEDEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
        (layers): ModuleList(
          (0-5): 6 x LEDEncoderLayer(
            (self_attn): LEDEncoderAttention(
              (longformer_self_attn): LEDEncoderSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (query_global): Linear(in_features=768, out_features=768, bias=True)
                (key_global): Linear(in_features=768, out_features=768, bias=True)
                (value_global): Linear(in_features=768, out_features=768, bias=True)
              )
  

In [13]:
# ref_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_model_id) 
# ref_model.to(device)

In [14]:
reward_model_directory = "reward_model"
reward_tokenizer_directory = "reward_model_tokenizer"

rm_model = AutoModelForSequenceClassification.from_pretrained(reward_model_directory).to(device)
rm_tokenizer = AutoTokenizer.from_pretrained(reward_tokenizer_directory)

In [15]:
split_ratio = 0.8  # 80% for training, 20% for evaluation
num_train_samples = int(split_ratio * len(dataset))
train_dataset = dataset.select(range(num_train_samples))
eval_dataset = dataset.select(range(num_train_samples, len(dataset)))


def tokenize_function(example):
    # Tokenize the prompt and store it as input_ids. Also return the response.
    return {
        "input_ids": policy_tokenizer(example["prompt"], return_tensors="pt", truncation=True, max_length=512)["input_ids"].squeeze(),
        "response": example["response"],
    }

train_dataset = train_dataset.map(tokenize_function, batched=False)
eval_dataset = eval_dataset.map(tokenize_function, batched=False)

In [16]:
train_dataset, eval_dataset

(Dataset({
     features: ['prompt', 'response', 'input_ids'],
     num_rows: 800
 }),
 Dataset({
     features: ['prompt', 'response', 'input_ids'],
     num_rows: 200
 }))

In [17]:
train_dataset[0]

{'prompt': "SUBREDDIT: r/relationships\nTITLE: I (f/22) have to figure out if I want to still know these girls or not and would hate to sound insulting\nPOST: Not sure if this belongs here but it's worth a try. \n\nBackstory:\nWhen I (f/22) went through my first real breakup 2 years ago because he needed space after a year of dating roand  it effected me more than I thought. It was a horrible time in my life due to living with my mother and finally having the chance to cut her out of my life. I can admit because of it was an emotional wreck and this guy was stable and didn't know how to deal with me. We ended by him avoiding for a month or so after going to a festival with my friends. When I think back I wish he just ended. So after he ended it added my depression I suffered but my friends helped me through it and I got rid of everything from him along with cutting contact. \n\nNow: Its been almost 3 years now and I've gotten better after counselling and mild anti depressants. My mothe

In [18]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [19]:
# Lets sample what the collator generates:
sample_data = [train_dataset[i] for i in range(3)]  # take first three examples
collated_data = collator(sample_data)
print(collated_data.keys())

dict_keys(['prompt', 'response', 'input_ids'])


In [20]:
learning_rate=1.41e-5
max_ppo_epochs=3
mini_batch_size=4
batch_size=16

DEFAULT_REJECTED_SUMMARY_TEXT = "This is a bad summary"

# Some initial values
output_min_length = 30
output_max_length = 150
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "temperature": 1.0,
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

max_ppo_steps = 100

In [21]:
config = PPOConfig(
    model_name=policy_model_id,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

In [22]:
ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,
                         #ref_model=ref_model,
                         tokenizer=policy_tokenizer,
                         dataset=train_dataset,
                         data_collator=collator)

In [23]:
import torch.nn.functional as F


def score_summaries(model, tokenizer, chosen_summary, rejected_summary):
    # Tokenize the inputs
    chosen_tokens = tokenizer(chosen_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    rejected_tokens = tokenizer(rejected_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

    chosen_tokens.to(device)
    rejected_tokens.to(device)

    # Get logits from the model
    with torch.no_grad():
        chosen_logits = model(**chosen_tokens).logits
        rejected_logits = model(**rejected_tokens).logits

    # Apply softmax to get probabilities
    chosen_probs = F.softmax(chosen_logits, dim=-1)
    rejected_probs = F.softmax(rejected_logits, dim=-1)

    # Assuming the positive class (indicating 'chosen' is good) is the second one
    chosen_score = chosen_probs[0][1].item()
    rejected_score = rejected_probs[0][1].item()

    # Extract logits for each summary
    chosen_logit = chosen_logits[0][1].item()
    rejected_logit = rejected_logits[0][1].item()

    return chosen_score, rejected_score, chosen_logit, rejected_logit

In [24]:
chosen_summary = "Water meter in another condo is not in our condo. What can we do legally to restore water to my condo complex?"
rejected_summary = "Go fix the problem."

chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

Chosen Score: 0.5161
Rejected Score: 0.4526
Chosen Logit: 0.3487
Rejected Logit: 0.0255


In [25]:
from tqdm import tqdm

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step >= max_ppo_steps: # Break when we reach max_steps.
        break

    prompt_tensors = batch["input_ids"]

    if isinstance(prompt_tensors, list) and all(isinstance(item, list) for item in prompt_tensors): # HACK!!! Check if original_prompt_tensors is a list of lists
        lengths = [len(seq) for seq in prompt_tensors] # Verify if sequences have fixed or variable length
        unique_lengths = set(lengths)

        if len(unique_lengths) > 1: # If sequences have variable lengths, pad them
            max_length = max(unique_lengths)
            original_prompt_tensors = [seq + [0] * (max_length - len(seq)) for seq in prompt_tensors]  # padding with zeros

        prompt_tensors = [torch.tensor(seq).to(device) for seq in prompt_tensors] # Convert original_prompt_tensors to individual tensors

    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        prompt_tensor = torch.tensor(prompt_tensor).to(device)
        max_new_tokens = output_length_sampler()
        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    batch["response"] = [policy_tokenizer.decode(r.squeeze()) for r in summary_tensors]

    chosen_summaries = batch["response"]
    rejected_summaries = [DEFAULT_REJECTED_SUMMARY_TEXT] * len(batch["response"])

    reward_tensors = []

    for chosen_summary, rejected_summary in zip(chosen_summaries, rejected_summaries):
        chosen_score, _, _, _ = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)
        reward_tensors.append(torch.tensor(chosen_score))

    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}') # Measures how different the policy's action distribution after the update is from the action distribution before the update. PPO tries to make these changes very small to avoid sudden changes.
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}') # This is the average return achieved by the agent. Higher is better.
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}') # Measures how much better an action is than the average action at a given state.
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]/tmp/ipykernel_55439/3451821770.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_tensor = torch.tensor(prompt_tensor).to(device)
Input ids are automatically padded from 493 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 431 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 195 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 221 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 274 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 450 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 504 to 1024 to be a multip

objective/kl: -9.676851732365321e-06
ppo/returns/mean: 0.5275627374649048
ppo/policy/advantages_mean: -0.002801092341542244
---------------------------------------------------------------------------------------------------


Input ids are automatically padded from 491 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 339 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 294 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 403 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 227 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 351 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 456 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 490 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 301 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 207 to 1024 to be a mult

objective/kl: -6.1851701736450195
ppo/returns/mean: 0.8105399012565613
ppo/policy/advantages_mean: 0.00224125268869102
---------------------------------------------------------------------------------------------------


Input ids are automatically padded from 342 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 355 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 429 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 284 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 236 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 332 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 381 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 283 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 336 to 1024 to be a multiple of `config.attention_window`: 1024
Input ids are automatically padded from 179 to 1024 to be a mult

...

objective/kl: 20.476886749267578
ppo/returns/mean: -1.948337435722351
ppo/policy/advantages_mean: -0.007052720990031958
---------------------------------------------------------------------------------------------------


49it [21:43, 21.69s/it]

objective/kl: 19.28260612487793
ppo/returns/mean: -1.769110083580017
ppo/policy/advantages_mean: 0.024180859327316284
---------------------------------------------------------------------------------------------------


Input ids are automatically padded from 388 to 1024 to be a multiple of `config.attention_window`: 1024
50it [22:05, 26.51s/it]

objective/kl: 18.823970794677734
ppo/returns/mean: -1.6166677474975586
ppo/policy/advantages_mean: 0.027625955641269684
---------------------------------------------------------------------------------------------------


In [28]:
stats

{'objective/kl': 18.823970794677734,
 'objective/kl_dist': array([24.216343, 19.108904, 21.07835 , 16.415909, 17.75911 , 14.864124,
        26.851608, 22.932692, 16.316277, 13.655459, 23.952072, 15.781115,
        18.548454, 11.432112, 10.116042, 28.154984], dtype=float32),
 'objective/logprobs': array([[-1.25168972e-05, -1.44583166e+00, -8.41871321e-01,
         -8.78231823e-01, -2.08248310e-02, -8.10619895e-06,
         -2.79246407e+01, -1.77062149e+01, -1.73297615e+01,
         -1.81244164e+01, -1.67996159e+01, -1.94089375e+01,
         -1.90715580e+01, -1.70691032e+01, -1.64233971e+01,
         -1.63964252e+01, -1.93038960e+01, -1.86627960e+01,
         -1.83176861e+01, -1.76473846e+01, -1.81938953e+01,
         -1.69870052e+01, -1.80033016e+01, -1.83171062e+01,
         -1.73901234e+01, -1.84903088e+01],
        [-1.33513513e-05, -1.82678267e-01, -1.44932792e-01,
         -3.91438752e-02, -3.55180586e-04, -2.02771928e-03,
         -2.76918132e-02, -2.46851967e-04, -2.89469872e+01,

In [29]:
ppo_trainer.model

AutoModelForSeq2SeqLMWithValueHead(
  (pretrained_model): LEDForConditionalGeneration(
    (led): LEDModel(
      (shared): Embedding(50265, 768, padding_idx=1)
      (encoder): LEDEncoder(
        (embed_tokens): Embedding(50265, 768, padding_idx=1)
        (embed_positions): LEDLearnedPositionalEmbedding(16384, 768)
        (layers): ModuleList(
          (0-5): 6 x LEDEncoderLayer(
            (self_attn): LEDEncoderAttention(
              (longformer_self_attn): LEDEncoderSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (query_global): Linear(in_features=768, out_features=768, bias=True)
                (key_global): Linear(in_features=768, out_features=768, bias=True)
                (value_global): Linear(in_features=768, out_features=768, bias=True)
              )
  

In [40]:
ppo_model_path = "ppo_model_final_new"
ppo_model_tokenizer = "ppo_model_tokenizer_final_new"

# ppo_trainer.save_model("ppo_model_path")
# Save the model
ppo_model.save_pretrained(ppo_model_path)

# Save the tokenizer
policy_tokenizer.save_pretrained(ppo_model_tokenizer)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 1024, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'repetition_penalty': 3.5, 'length_penalty': 0.8, 'no_repeat_ngram_size': 3}


('ppo_model_tokenizer_final_new/tokenizer_config.json',
 'ppo_model_tokenizer_final_new/special_tokens_map.json',
 'ppo_model_tokenizer_final_new/vocab.json',
 'ppo_model_tokenizer_final_new/merges.txt',
 'ppo_model_tokenizer_final_new/added_tokens.json',
 'ppo_model_tokenizer_final_new/tokenizer.json')

In [41]:
# ppo_saved_model_path = "ppo_model_20Epochs_new"
# tokenizer_path = "ppo_model_tokenizer_new"
ppo_model = AutoModelForSeq2SeqLM.from_pretrained(ppo_model_path)
policy_tokenizer = AutoTokenizer.from_pretrained(ppo_model_tokenizer)

Some weights of the model checkpoint at ppo_model_final_new were not used when initializing LEDForConditionalGeneration: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing LEDForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LEDForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
# def generate_summary(prompt: str, model, tokenizer, generation_kwargs, output_length_sampler) -> str:
#     """
#     Generate a summary for a given prompt using a trained policy model.

#     Args:
#     - prompt (str): The input text for which a summary needs to be generated.
#     - model: The trained policy model.
#     - tokenizer: The tokenizer used for the policy model.
#     - generation_kwargs (dict): Arguments used for response generation.
#     - output_length_sampler (func): Function to sample the length of the output.

#     Returns:
#     - str: Generated summary.
#     """

#     # Tokenize the prompt
#     prompt_tensor = tokenizer.encode(prompt, return_tensors='pt').to(device)

#     # Ensure it's only one tensor and check its shape
#     assert prompt_tensor.dim() == 2, f"Unexpected tensor shape: {prompt_tensor.shape}"

#     # Set the generation arguments
#     max_new_tokens = output_length_sampler()
#     generation_kwargs["max_new_tokens"] = max_new_tokens

#     # Generate a summary
#     summary_tensor = model.generate(input_ids=prompt_tensor, **generation_kwargs)

#     # Decode and return the summary
#     summary = tokenizer.decode(summary_tensor[0], skip_special_tokens=True)
#     return summary

In [31]:
testing_sample = pd.read_csv('testing_samples.csv')
testing_sample

,prompt,label
0,SUBREDDIT: r/AskReddit\nTITLE: I need your hel...,American Family Insurance is screwing me with ...
1,SUBREDDIT: r/relationships\nTITLE: My boyfrien...,Boyfriend of 3 years started a business withou...
2,SUBREDDIT: r/AskReddit\nTITLE: Can someone hel...,Grandpa had a light bulb he could light up by ...
3,SUBREDDIT: r/travel\nTITLE: If I don't do this...,"I'm an American, bored with my career, wanting..."
4,SUBREDDIT: r/tifu\nTITLE: TIFU By Showing My H...,Made a bet with teacher to watch Vader vs Hitl...
5,SUBREDDIT: r/dating_advice\nTITLE: Should I [1...,Have a bit of a crush on a guy who I see every...
6,SUBREDDIT: r/relationship_advice\nTITLE: When ...,"If we both know we like each other, and have r..."
7,SUBREDDIT: r/relationships\nTITLE: I [18 M] ha...,Interested in a girl i sit with next to in cla...
8,SUBREDDIT: r/Advice\nTITLE: Freaking out about...,Freaking out about college being too much and ...
9,SUBREDDIT: r/personalfinance\nTITLE: 25 y/o lo...,forces out of home. I have $400 and $6000 debt...


In [34]:
pipe = pipeline("summarization", model=policy_model, tokenizer=policy_tokenizer) #max_length=350, num_return_sequences=1
Base_model_summary = []
for i in testing_sample['prompt']:
    output = pipe(i,temperature =  1.0, min_length = 5, top_k = 0.0, top_p = 1.0, do_sample =  True, max_length=150)
    Base_model_summary.append(output)

In [35]:
Base_model_summary 

[[{'summary_text': 'A Redditch is asking Reddit for help finding his insurance company, American Family Insurance. He explains that he was involved in a car accident where the other driver was at fault. The insurance company arranged for him to get a rental car and pay for auto body repair at the shop my dealer recommended. However, when he picked up the rental car, the insurance company paid for it, he had to sign for the coverage that the rental company (Enterprise) offers, which is $13 a day. This means that he will have to buy a new car every 2 weeks. Plus, there\'s no way this insurance company can "pay for rental insurance when their client is at fault." Translation: I don\'t have full liability insurance, so'}],
 [{'summary_text': 'A Reddit user has been asking a question about relationships: Why is it that his "part-time grad student" boyfriend of 3 years keeps making huge decisions without communicating with me [23 F] at all, is this normal? The redditor explains that he\'s ju

In [36]:
pipe = pipeline("summarization", model=ppo_model, tokenizer=policy_tokenizer) #max_length=350, num_return_sequences=1
PPO_model_summary = []
for i in testing_sample['prompt']:
    output = pipe(i,temperature =  1.0, min_length = 5, top_k = 0.0, top_p = 1.0, do_sample = True, max_length=150)
    PPO_model_summary.append(output)
PPO_model_summary

The model 'AutoModelForSeq2SeqLMWithValueHead' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].


[[{'summary_text': 'Does my car need repair?'}],
 [{'summary_text': 'Does this normal?'}],
 [{'summary_text': 'Can someone please help my dad?'}],
 [{'summary_text': "I'm looking for advice, help, reassurance, dissuasion or a little bit of each on a major life decision."}],
 [{'summary_text': 'Why would you need those movies?'}],
 [{'summary_text': "Should I, who went on a couple dead end dates with my friend's sister?"}],
 [{'summary_text': 'Does it alright to just ask here to be "more than friends"?'}],
 [{'summary_text': 'I am talking to a student in my class.'}],
 [{'summary_text': 'What do I do?'}],
 [{'summary_text': 'Does a career change make a smart idea?'}]]

In [38]:
testing_sample['base_model'] = Base_model_summary
testing_sample['PPO'] = PPO_model_summary

rlhf_result = testing_sample

rlhf_result.to_csv('rlhf_result_new.csv')

In [74]:
# Base_model_summary = []
# for i in testing_sample['prompt']:
#     prompt = "summarize: " + i
#     generated_summary = generate_summary(prompt, policy_model, policy_tokenizer, generation_kwargs, output_length_sampler)
#     Base_model_summary.append(generated_summary)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [76]:
# ppo_model_summary = []
# for i in testing_sample['prompt']:
#     prompt = "summarize: " + i
#     generated_summary = generate_summary(prompt, ppo_model, policy_tokenizer, generation_kwargs, output_length_sampler)
#     ppo_model_summary.append(generated_summary)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [127]:
# pipe = pipeline("summarization", model=ppo_model, tokenizer=policy_tokenizer) #max_length=350, num_return_sequences=1
# PPO_model_summary = []
# for i in testing_sample['prompt']:
#     output = pipe(i,temperature =  1.0, min_length = 5, top_k = 0.0, top_p = 1.0, do_sample =  True, max_length=150)
#     PPO_model_summary.append(output)

In [128]:
# PPO_model_summary

[[{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh yeah.'}],
 [{'summary_text': 'Oh, yeah'}],
 [{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh yeah.'}],
 [{'summary_text': 'Oh, yeah.'}],
 [{'summary_text': 'Oh, yeah.'}]]